In [85]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from statsmodels.tsa.arima_model import AR, ARMA, ARIMA

def eda(dataframe):
    print "missing values \n", dataframe.isnull().sum()
    print "dataframe index \n", dataframe.index
    print "dataframe types \n", dataframe.dtypes
    print "dataframe shape \n", dataframe.shape
    print "dataframe describe \n", dataframe.describe()
    print "dataframe duplicates \n", dataframe.duplicated().sum() 
    for item in dataframe:
        print item
        print dataframe[item].nunique()
        

def eda_helper(df):
    dict_list = []
    for col in df.columns:
        data = df[col]
        dict_ = {}
        # The null count for a column. Columns with no nulls are generally more interesting
        dict_.update({"null_count" : data.isnull().sum()})
        # Counting the unique values in a column
        # This is useful for seeing how interesting the column might be as a feature
        dict_.update({"unique_count" : len(data.unique())})
        # Finding the types of data in the column
        # This is useful for finding out potential problems with a column having strings and ints
        dict_.update({"data_type" : set([type(d).__name__ for d in data])})
        #dict_.update({"score" : match[1]})
        dict_list.append(dict_)
    eda_df = pd.DataFrame(dict_list)
    eda_df.index = df.columns
    #eda_df = eda_df.sort_values(['null_count','unique_count'], ascending=[True, False])
    return eda_df

In [2]:
data = pd.read_csv('/users/andresgonzalez/Desktop/2016_campaign_spending.csv')

In [7]:
data.head()

,filing_id,payee,street,city,state,date,amount,cat,just_year,fec_committee_id,committee_name
0,1123057,"Riddems Marketing, Inc.",6151 Miramar Pkwy,Miramar,FL,2016-10-07,10000.00,Generic Radio Advertisement,2016.0,C00005561,Democratic Executive Committee of Florida
1,1123057,"Progressive Organizing Solutions, LLC",100 E Grand Ave,Des Moines,IA,2016-10-11,489726.67,GOTV Consulting,2016.0,C00005561,Democratic Executive Committee of Florida
2,1101705,Wisconsin Department of Workforce Development,201 E Washington Ave,Madison,WI,2016-03-31,824.47,Wisconsin unemployment,2016.0,C00019331,Democratic Party of Wisconsin
3,1101705,Internal Revenue Service,IRS Service Ctr,Kansas City,MO,2016-03-31,7.12,federal unemployment,2016.0,C00019331,Democratic Party of Wisconsin
4,1101705,Internal Revenue Service,IRS Service Ctr,Kansas City,MO,2016-03-31,1248.84,FICA Match,2016.0,C00019331,Democratic Party of Wisconsin


In [8]:
eda(data)

missing values 
filing_id              0
payee                  0
street               582
city                 496
state                531
date                   0
amount                 0
cat                 6033
just_year              0
fec_committee_id       0
committee_name         0
dtype: int64
dataframe index 
RangeIndex(start=0, stop=169393, step=1)
dataframe types 
filing_id             int64
payee                object
street               object
city                 object
state                object
date                 object
amount              float64
cat                  object
just_year           float64
fec_committee_id     object
committee_name       object
dtype: object
dataframe shape 
(169393, 11)
dataframe describe 
          filing_id        amount  just_year
count  1.693930e+05  1.693930e+05   169393.0
mean   1.107874e+06  5.466811e+03     2016.0
std    5.167441e+03  1.303434e+05        0.0
min    1.101705e+06 -3.218749e+05     2016.0
25%    1.104679e+06  3.5

In [9]:
eda_helper(data)

,data_type,null_count,unique_count
filing_id,{int64},0,4301
payee,{str},0,34603
street,"{float, str}",582,45808
city,"{float, str}",496,7231
state,"{float, str}",531,71
date,{str},0,318
amount,{float64},0,46440
cat,"{float, str}",6033,20321
just_year,{float64},0,1
fec_committee_id,{str},0,3875


In [98]:
headers=['id', 'payee', 'address', 'city', 'state', 'date', 'amount', 'concept']

expenditures = pd.read_csv('/users/andresgonzalez/Desktop/expenditures.csv', header=None, names=headers)

In [4]:
expenditures.head()

,id,payee,address,city,state,date,amount,concept
0,189180,Patrick Mooney & Assoc,20560 Wake Terrace,Sterling,VA,2005-08-23,3119.29,CREATIVE FEES
1,189180,Capitol Hill Lists,"337 S. Milledge Avenue, Ste. 204",Athens,GA,2005-08-23,1200.00,POSTAGE
2,189180,Mrs. Julie Reichert,P.O. Box 53322,Bellevue,WA,2005-08-22,263.41,SEE BELOW
3,189180,C&B Awards,16204 SE 272nd Street,Covington,WA,2005-08-22,263.41,GIFTS
4,189180,B.W. Crystaline & Associates,6830 NE Bothell Way,Kenmore,WA,2005-09-06,1614.00,BOOKKEEPING


In [12]:
expenditures.shape

(49222681, 8)

In [13]:
eda(expenditures)

missing values 
189180                         0
Patrick Mooney & Assoc         0
20560 Wake Terrace         38659
Sterling                   21114
VA                         19284
2005-08-23                   581
3119.29                        0
CREATIVE FEES             517177
dtype: int64
dataframe index 
RangeIndex(start=0, stop=49222681, step=1)
dataframe types 
189180                      int64
Patrick Mooney & Assoc     object
20560 Wake Terrace         object
Sterling                   object
VA                         object
2005-08-23                 object
3119.29                   float64
CREATIVE FEES              object
dtype: object
dataframe shape 
(49222681, 8)
dataframe describe 
             189180       3119.29
count  4.922268e+07  4.922268e+07
mean   9.117422e+05  5.655043e+02
std    2.476510e+05  3.226019e+04
min    2.709300e+04 -9.071482e+06
25%    8.412410e+05  5.000000e+00
50%    1.024608e+06  1.500000e+01
75%    1.074053e+06  5.000000e+01
max    1.134908e+06  

In [14]:
eda_helper(expenditures)

,data_type,null_count,unique_count
189180,{int64},0,294789
Patrick Mooney & Assoc,{str},0,749259
20560 Wake Terrace,"{float, str}",38659,941295
Sterling,"{float, str}",21114,43253
VA,"{float, str}",19284,291
2005-08-23,"{float, str}",581,6187
3119.29,{float64},0,615512
CREATIVE FEES,"{float, str}",517177,949575


In [99]:
expenditures.date = expenditures.date.astype(str)

In [100]:
expenditures['datetime']= pd.to_datetime(expenditures.date, format='%y-%m-%d', errors='ignore')

In [101]:
expenditures.date[5372783] = '2012-03-01'
expenditures.date[5411339] = '2012-05-15'
expenditures.date[14161479] = '2014-03-01'
expenditures.date[21537454] = '2015-04-01'

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda/lib/python2.7/site-packa

In [30]:
expenditures.index = expenditures['datetime'] 

In [33]:
expenditures.head()

,id,payee,address,city,state,date,amount,concept,datetime
datetime,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,20560 Wake Terrace,Sterling,VA,2005-08-23,3119.29,CREATIVE FEES,2005-08-23
2005-08-23,189180,Capitol Hill Lists,"337 S. Milledge Avenue, Ste. 204",Athens,GA,2005-08-23,1200.00,POSTAGE,2005-08-23
2005-08-22,189180,Mrs. Julie Reichert,P.O. Box 53322,Bellevue,WA,2005-08-22,263.41,SEE BELOW,2005-08-22
2005-08-22,189180,C&B Awards,16204 SE 272nd Street,Covington,WA,2005-08-22,263.41,GIFTS,2005-08-22
2005-09-06,189180,B.W. Crystaline & Associates,6830 NE Bothell Way,Kenmore,WA,2005-09-06,1614.00,BOOKKEEPING,2005-09-06


In [52]:
expclean = expenditures.datetime

In [53]:
column_1 = pd.to_datetime(expclean)

In [55]:


dtime = pd.DataFrame({"year": column_1.dt.year, "month": column_1.dt.month, "day": column_1.dt.day, "dayofyear": column_1.dt.dayofyear, "week": column_1.dt.week, "dayofweek": column_1.dt.dayofweek, "weekday": column_1.dt.weekday, "quarter": column_1.dt.quarter})

In [56]:
dtime.head()

,day,dayofweek,dayofyear,month,quarter,week,weekday,year
datetime,,,,,,,,
2005-08-23,23.0,1.0,235.0,8.0,3.0,34.0,1.0,2005.0
2005-08-23,23.0,1.0,235.0,8.0,3.0,34.0,1.0,2005.0
2005-08-22,22.0,0.0,234.0,8.0,3.0,34.0,0.0,2005.0
2005-08-22,22.0,0.0,234.0,8.0,3.0,34.0,0.0,2005.0
2005-09-06,6.0,1.0,249.0,9.0,3.0,36.0,1.0,2005.0


In [57]:
cleandf = pd.concat((expenditures, dtime), axis=1)

In [58]:
print cleandf.shape
cleandf.head()

(49222682, 17)


,id,payee,address,city,state,date,amount,concept,datetime,day,dayofweek,dayofyear,month,quarter,week,weekday,year
datetime,,,,,,,,,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,20560 Wake Terrace,Sterling,VA,2005-08-23,3119.29,CREATIVE FEES,2005-08-23,23.0,1.0,235.0,8.0,3.0,34.0,1.0,2005.0
2005-08-23,189180,Capitol Hill Lists,"337 S. Milledge Avenue, Ste. 204",Athens,GA,2005-08-23,1200.00,POSTAGE,2005-08-23,23.0,1.0,235.0,8.0,3.0,34.0,1.0,2005.0
2005-08-22,189180,Mrs. Julie Reichert,P.O. Box 53322,Bellevue,WA,2005-08-22,263.41,SEE BELOW,2005-08-22,22.0,0.0,234.0,8.0,3.0,34.0,0.0,2005.0
2005-08-22,189180,C&B Awards,16204 SE 272nd Street,Covington,WA,2005-08-22,263.41,GIFTS,2005-08-22,22.0,0.0,234.0,8.0,3.0,34.0,0.0,2005.0
2005-09-06,189180,B.W. Crystaline & Associates,6830 NE Bothell Way,Kenmore,WA,2005-09-06,1614.00,BOOKKEEPING,2005-09-06,6.0,1.0,249.0,9.0,3.0,36.0,1.0,2005.0


In [68]:
final = cleandf[cleandf.year >=2004]

In [69]:
final.drop(['address', 'date', 'concept', 'dayofweek', 'dayofyear', 'quarter', 'week' , 'weekday'], axis = 1, inplace=True)
final.head()

/anaconda/lib/python2.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,id,payee,city,state,amount,datetime,day,month,year
datetime,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,Sterling,VA,3119.29,2005-08-23,23.0,8.0,2005.0
2005-08-23,189180,Capitol Hill Lists,Athens,GA,1200.00,2005-08-23,23.0,8.0,2005.0
2005-08-22,189180,Mrs. Julie Reichert,Bellevue,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-08-22,189180,C&B Awards,Covington,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-09-06,189180,B.W. Crystaline & Associates,Kenmore,WA,1614.00,2005-09-06,6.0,9.0,2005.0


In [70]:
final.to_csv('finaldf.csv')

In [71]:
df04 = final[final.year == 2004]

In [74]:
df08 = final[final.year == 2008]

In [75]:
df12 = final[final.year == 2012]

In [76]:
df16 = final[final.year == 2016]

In [77]:
df04.to_csv('df04.csv')
df08.to_csv('df08.csv')
df12.to_csv('df12.csv')
df16.to_csv('df16.csv')

In [106]:
df16.head()

,id,payee,city,state,amount,datetime,day,month,year
datetime,,,,,,,,,
2016-10-07,1123057,"Riddems Marketing, Inc.",Miramar,FL,10000.00,2016-10-07,7.0,10.0,2016.0
2016-10-11,1123057,"Progressive Organizing Solutions, LLC",Des Moines,IA,489726.67,2016-10-11,11.0,10.0,2016.0
2016-03-31,1101705,Wisconsin Department of Workforce Development,Madison,WI,824.47,2016-03-31,31.0,3.0,2016.0
2016-03-31,1101705,Internal Revenue Service,Kansas City,MO,7.12,2016-03-31,31.0,3.0,2016.0
2016-03-31,1101705,Internal Revenue Service,Kansas City,MO,1248.84,2016-03-31,31.0,3.0,2016.0


In [81]:
df16f = pd.DataFrame(df16.amount.groupby(df16.index).sum())

In [82]:
df16f.head()

,amount
datetime,
2016-01-01,2535995.60
2016-01-02,2611207.49
2016-01-03,1222058.10
2016-01-04,28702959.30
2016-01-05,7723036.62


In [89]:
df12f = pd.DataFrame(df12.amount.groupby(df12.index).sum())

In [108]:
df12.datetime = df12.datetime.astype(str)
df12.set_index('datetime')
df12.head()


/anaconda/lib/python2.7/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,id,payee,city,state,amount,datetime,day,month,year
datetime,,,,,,,,,
2012-06-14,1102169,Tallahassee Urban League,Tallahassee,FL,1000.0,2012-06-14,14.0,6.0,2012.0
2012-05-30,1102169,Tallahassee Union,Tallahassee,FL,1000.0,2012-05-30,30.0,5.0,2012.0
2012-06-21,1102169,Tallahassee Democrat,Tallahassee,FL,700.0,2012-06-21,21.0,6.0,2012.0
2012-04-18,1102169,Richir Outreach,Raleigh,NC,360.0,2012-04-18,18.0,4.0,2012.0
2012-04-12,1102169,Rays Odds Jobs,Tallahassee,FL,400.0,2012-04-12,12.0,4.0,2012.0


In [105]:
df12f.index = df12f.index.map(str)
# df12f.index = df12f.index.map(unicode)

AttributeError: 'Index' object has no attribute 'index'

In [114]:
df12f = pd.DataFrame(df12.amount.groupby(df12.index).sum())
type(df12f.in)

str

In [115]:
expenditures.head()

,id,payee,address,city,state,date,amount,concept,datetime
0,189180,Patrick Mooney & Assoc,20560 Wake Terrace,Sterling,VA,2005-08-23,3119.29,CREATIVE FEES,2005-08-23
1,189180,Capitol Hill Lists,"337 S. Milledge Avenue, Ste. 204",Athens,GA,2005-08-23,1200.00,POSTAGE,2005-08-23
2,189180,Mrs. Julie Reichert,P.O. Box 53322,Bellevue,WA,2005-08-22,263.41,SEE BELOW,2005-08-22
3,189180,C&B Awards,16204 SE 272nd Street,Covington,WA,2005-08-22,263.41,GIFTS,2005-08-22
4,189180,B.W. Crystaline & Associates,6830 NE Bothell Way,Kenmore,WA,2005-09-06,1614.00,BOOKKEEPING,2005-09-06


In [116]:
final.head()

,id,payee,city,state,amount,datetime,day,month,year
datetime,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,Sterling,VA,3119.29,2005-08-23,23.0,8.0,2005.0
2005-08-23,189180,Capitol Hill Lists,Athens,GA,1200.00,2005-08-23,23.0,8.0,2005.0
2005-08-22,189180,Mrs. Julie Reichert,Bellevue,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-08-22,189180,C&B Awards,Covington,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-09-06,189180,B.W. Crystaline & Associates,Kenmore,WA,1614.00,2005-09-06,6.0,9.0,2005.0


In [122]:
pre = final[final.year <2010]
pre.head()

,id,payee,city,state,amount,datetime,day,month,year
datetime,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,Sterling,VA,3119.29,2005-08-23,23.0,8.0,2005.0
2005-08-23,189180,Capitol Hill Lists,Athens,GA,1200.00,2005-08-23,23.0,8.0,2005.0
2005-08-22,189180,Mrs. Julie Reichert,Bellevue,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-08-22,189180,C&B Awards,Covington,WA,263.41,2005-08-22,22.0,8.0,2005.0
2005-09-06,189180,B.W. Crystaline & Associates,Kenmore,WA,1614.00,2005-09-06,6.0,9.0,2005.0


In [125]:
pre = pre[pre.year>2005]
pre.amount.sum()

5018083681.050003

In [126]:
post = final[final.year >= 2010]
post = post[post.year <=2016]
post.amount.sum()

17824036447.571224

In [132]:
statessumpre = pre.groupby('state').amount.sum()


In [133]:
statessumpre[statessumpre.index == 'VA']

state
VA    7.652284e+08
Name: amount, dtype: float64

In [134]:
statessumpost = post.groupby('state').amount.sum()
statessumpost[statessumpost.index == 'VA']


state
VA    2.046702e+09
Name: amount, dtype: float64

In [135]:
print statessumpre[statessumpre.index == 'FL']
print statessumpost[statessumpost.index == 'FL']

state
FL    1.445323e+08
Name: amount, dtype: float64
state
FL    5.881673e+08
Name: amount, dtype: float64


In [136]:
print statessumpre[statessumpre.index == 'OH']
print statessumpost[statessumpost.index == 'OH']

state
OH    1.914326e+08
Name: amount, dtype: float64
state
OH    5.511806e+08
Name: amount, dtype: float64


In [137]:
print statessumpre[statessumpre.index == 'CA']
print statessumpost[statessumpost.index == 'CA']

state
CA    2.200197e+08
Name: amount, dtype: float64
state
CA    9.557479e+08
Name: amount, dtype: float64


In [ ]:
cmt = pd.read_csv('/users/andresgonzalez/Desktop/filings.csv')

In [ ]:
cmtdict = dict(zip(cmt.filing_id, cmt.committee_name))

In [6]:
expenditures.dropna(inplace=True)

In [9]:
eda_helper(expenditures)

,data_type,null_count,unique_count
id,{int64},0,273757
payee,{str},0,724402
address,{str},0,921495
city,{str},0,41532
state,{str},0,279
date,{str},0,6167
amount,{float64},0,611921
concept,{str},0,944960


In [42]:
expenditures.date = expenditures.date.astype(str)

In [41]:
expenditures.head()

,id,payee,address,city,state,date,amount,concept,datetime
datetime,,,,,,,,,
2005-08-23,189180,Patrick Mooney & Assoc,20560 Wake Terrace,Sterling,VA,2005-08-23,3119.29,CREATIVE FEES,2005-08-23
2005-08-23,189180,Capitol Hill Lists,"337 S. Milledge Avenue, Ste. 204",Athens,GA,2005-08-23,1200.00,POSTAGE,2005-08-23
2005-08-22,189180,Mrs. Julie Reichert,P.O. Box 53322,Bellevue,WA,2005-08-22,263.41,SEE BELOW,2005-08-22
2005-08-22,189180,C&B Awards,16204 SE 272nd Street,Covington,WA,2005-08-22,263.41,GIFTS,2005-08-22
2005-09-06,189180,B.W. Crystaline & Associates,6830 NE Bothell Way,Kenmore,WA,2005-09-06,1614.00,BOOKKEEPING,2005-09-06


In [18]:
expenditures['datetime']= pd.to_datetime(expenditures.date, format='%y-%m-%d', errors='ignore')